# Tutorial para analizar los read partidos en las deleciones

## 1° Buscar los archivos "sam" de la muestra 413 los cuales tienen deleciones detectadas


Debemos ingresar al cluster de Langebio con el usuario segun nos correspoda (ssh usuario@ip)
$ cd /LUSTRE/usuario/aherrera/covid/abril/alineamientosMal/celia-sam

Corrimos el archivo 413.sam con el siguiente script

 $ cat 413.sam | cut -f1,4 | sort -k3 -n -r > name-reads
 
cat : abre el archivo
cut -f1,4 = del archivo recorta las columnas 1 y 4
sort = ordena los datos de las dos columnas
* -k3 = ordena los datos por la columna 3 (deberia ser por la segunda columna, pero tenemos un espacio en la linea y hacer que bash se confunda).
* -n = los ordena de forma numerica
* -r = al mismo tiempo pone los datos del mayor al menor
    
nota: recuerda que cuando tienes un espacio en la linea, bash se confunde y lo toma como otra columna. Verifica que tu linea no tenga espacios.

# output: 

[aherrera@mazorka celia-sam]$ cat 413.sam | cut -f1,4 | sort -k3 -n -r | head

NB501110:185:H7L72AFX2:3:11507:21445:1375 1:N:0:TAAGGCGA+AGAGGATA       29835
NB501110:185:H7L72AFX2:4:11507:1091:3775 2:N:0:TAAGGCGA+AGAGGATA        29818
NB501110:185:H7L72AFX2:4:11507:1091:3775 1:N:0:TAAGGCGA+AGAGGATA        29818
NB501110:185:H7L72AFX2:4:11506:26472:3781 2:N:0:TAAGGCGA+AGAGGATA       29818
NB501110:185:H7L72AFX2:4:11506:26472:3781 1:N:0:TAAGGCGA+AGAGGATA       29818
NB501110:185:H7L72AFX2:3:21511:10348:14439 2:N:0:TAAGGCGA+AGAGGATA      29818
NB501110:185:H7L72AFX2:3:21511:10348:14439 1:N:0:TAAGGCGA+AGAGGATA      29818
NB501110:185:H7L72AFX2:4:21404:14848:1270 2:N:0:TAAGGCGA+AGAGGATA       29814
NB501110:185:H7L72AFX2:4:21404:14848:1270 1:N:0:TAAGGCGA+AGAGGATA       29814
NB501110:185:H7L72AFX2:2:21310:26034:3472 2:N:0:TAAGGCGA+AGAGGATA       29808

In [ ]:
 ## 2° Identifica la posicion de la deleción:
* Inicia la delecion: 27875 - 100 posiciones
* Termina la delecion: 28233 + 100 posiciones

## 3° Obten una lista de los reads que hagan match 150 pb antes del inicio de la delecion

$ awk '$3 >= 27775 && $3 <= 27925 ' name-reads > antes-del

este script te dice que de la columna 3 tome los valores que sean mayor igual a 27775, pero que tambien tome los valores que sean menor igual a 27925.

## 4° obten una lista de los reads que hagan match 150 pb despues del final de la delecion

$ awk '$3 >= 28233 && $3 <= 28383 ' name-reads > despues-del

#nota: recuerda que cuando tienes un espacio en la linea, bash se confunde y lo toma como otra columna. Verifica que tu linea no tenga espacios.

## 5° compara las dos listas entre si 

Unir los archivos "antes-del" y "despues-del"

$ cat antes-del despues-del > all-del

Ordenar y contar los reads repetidos

$ cat all-del | sort | uniq -c

cat : abre el archivo all-del 
sort: ordena los archivos
uniq -c : los archivos que esten repetidos los une en uno solo y al fina cuenta cuantos hay repetidos.

## 6° si hace match el metodo no funciona para detectar deleciones

[aherrera@mazorka celia-sam]$ cat all-del | sort | uniq -c | head

      1 NB501110:185:H7L72AFX2:1:11101:13093:13622 1:N:0:TAAGGCGA+AGAGGATA      27792
      1 NB501110:185:H7L72AFX2:1:11101:13093:13622 2:N:0:TAAGGCGA+AGAGGATA      27792
      1 NB501110:185:H7L72AFX2:1:11101:13597:8716 2:N:0:TAAGGCGA+AGAGGATA       28234
      1 NB501110:185:H7L72AFX2:1:11101:14301:1188 1:N:0:TAAGGCGA+AGAGGATA       28255
      1 NB501110:185:H7L72AFX2:1:11101:14301:1188 2:N:0:TAAGGCGA+AGAGGATA       28305
      1 NB501110:185:H7L72AFX2:1:11101:15710:10990 1:N:0:TAAGGCGA+AGAGGATA      28254
      1 NB501110:185:H7L72AFX2:1:11101:15710:10990 2:N:0:TAAGGCGA+AGAGGATA      28254
      1 NB501110:185:H7L72AFX2:1:11101:16283:4812 1:N:0:TAAGGCGA+AGAGGATA       28267
      1 NB501110:185:H7L72AFX2:1:11101:16283:4812 2:N:0:TAAGGCGA+AGAGGATA       28305
      1 NB501110:185:H7L72AFX2:1:11101:1895:1113 1:N:0:TAAGGCGA+AGAGGATA        28234




Conclusion: No hace match con los reads, posiblemente el metodo funciona para detectar deleciones.

Ahora entonces intentaremos por blast 

In [ ]:

# encontrar inicio de match de read y guardarlo en una variable que vas a usar en awk
# buscar con awk los que sean mayores a 27000
# buscar con awk los que sean menores que 28200

In [73]:
%%bash
ls blast-deleciones/

#Deleciones1.txt: es un archivo que obtuvimos del blast y que hace match en dos regiones del genoma
#Deleciones2.txt: es un archivo que obtuvimos del blast y que hace match en una regione del genoma


413blast
data
db
deleciones1-blast.out
deleciones2-blast.out
deleciones.sh
deleciones.txt
lista
nodelecion.txt


In [57]:
%%bash
#Hacer un escript que me busque en el archivo un numero que haga match en la region 27
grep -l 'Sbjct* *277' blast-deleciones/*.out >> 27mil
#grep -l 'Sbjct* *277' blast-deleciones/deleciones2.txt >> 27mil
#Hacer un escript que me busque en el archivo un numero que haga match en la region 28 mil
grep -l 'Sbjct* *282' blast-deleciones/*.out >> 28mil
#grep -l 'Sbjct* *282' blast-deleciones/deleciones2.txt >> 28mil
#grep 'Sbjct* *282' blast-deleciones/deleciones1.txt >> 28mil
#Hacer un escript que me busque en el archivo un numero que haga match en ambas regiones

In [66]:
%%bash
cat 28mil | sort | uniq > 28mil-uniq

In [67]:
%%bash
cat 27mil | sort | uniq > 27mil-uniq

In [69]:
cat 27mil-uniq 28mil-uniq | sort | uniq -c | grep '2 '

      2 blast-deleciones/deleciones1-blast.out


# Este comando funciona para poder separar un archivo multifasta y generar varios archivos fasta

$ split ../413reads.fasta -l 2 preblast- --verbose

output:

creando archivo 'preblast-zpwy'
creando archivo 'preblast-zpwz'
creando archivo 'preblast-zpxa'
creando archivo 'preblast-zpxb'
creando archivo 'preblast-zpxc'
creando archivo 'preblast-zpxd'
creando archivo 'preblast-zpxe'
creando archivo 'preblast-zpxf'
creando archivo 'preblast-zpxg'


In [ ]:

../413reads.fasta: archivo multifasta
-l 2: esta opción le dices que corte el archivo por las dos filas
preblast- : el es prefijo para los nuevos archivos
--verbose : es para poder ver el proceso de split

## Hicimos blastn a todos los archivos fasta que se generaron 

 cat lista | while read line ; do echo $line; blastn -query ../data/413-del/413split/$line -db /media/betterlab/Datos01/abel/Deleciones-Mex/blast-deleciones/db/reference-covid19.fasta -out $line.txt; done
 
 el archivo lista estan los nombres de los archivos que generamos cuando utilizamos split, e hicimos un while para hacer los blast de un solo golpe
 

## Buscamos los reads que cumplieran con el siguiente numero 277

 grep -l 'Sbjct* *277' *.txt >> 27mil
 
 note:
 -l: te va a imprimir el nombre del archivo de lo que estas buscando
 >>: Este te sirve para redireccionar los archivos y no te lo sobreescriba
 

## Buscamos los reads que cumplieran con el siguiente numero 282

grep -l 'Sbjct* *282' *.txt >> 28mil

## Con este comando lo que hicimos fue eliminar los nombres dobles y evitar equivocaciones

cat 28mil | sort | uniq > 28mil-uniq
 2195  cat 27mil | sort | uniq > 27mil-uniq

In [74]:
## Por ultimo contamos los read que estan partidos antes y despues de la delecion

In [83]:
%%bash
cd blast-deleciones/413blast
cat 27mil-uniq 28mil-uniq | sort | uniq -c | grep '2 ' | wc -l


934


Conclusion: existen 934 reads que estan partidos antes de la delecion como despues de la delecion

## 2° Haremos lo mismo que los anterior, pero sera con un archivo que no tiene deleción
